In [1]:
import zarr
import iris, iris_grib
import numpy as np
import xarray as xr
import sys, os
from datetime import datetime,timedelta,date
from numcodecs import Blosc, Zstd, LZ4
import time
import zarr_utilsv4 as zutils

In [2]:
dataLoc = '/uufs/chpc.utah.edu/common/home/horel-group4/tmccorkle/hrrr_conus/'
dataStore = '/uufs/chpc.utah.edu/common/home/horel-group4/tmccorkle/hrrr_conus/'

month = '2021'
day = ['0314']
        
nDay = len(day)

idx = [0,48,54,72,90,108,126,162,180,198,216,234,252,288,306,324,342,
           360,378,414,432,450,468,486,504]

for d in range(0,nDay):
    dayList = month+day[d]
    print(dayList)
    forecastDir = dataLoc+dayList+'/forecasts/'
    writeTo = dataStore+dayList+'/'

    filesFcst = sorted(os.listdir(forecastDir))

    filepaths = [os.path.join(forecastDir, fileName) for fileName in filesFcst]

                                            
    for hr in range(0,1):
        t1 = time.time()
        print('model run',hr)
        hour = '{:02d}'.format(hr)

        storeFcst = zarr.DirectoryStore(writeTo+dayList+'_'+hour+'z_fcst.zarr')
        root_fcst = zarr.group(store=storeFcst)#,overwrite=True)
        
        
        a = idx[hr]
        z = idx[hr+1]

        hrs = (z-a)

        cubes = iris_grib.load_cubes(filepaths[a:z])
        cube_list = list(cubes)

        zarr_group = zutils.get_idx_forecast(dayList,hrs)
        cubelists = zutils.rename_cubes(cube_list, zarr_group)


        cubelists = iris.cube.CubeList(cubelists)
        new_cube = cubelists.merge(unique = False)
        cubelist, diffVar = zutils.update_coords(new_cube)
        
        if hr == 0 or hr == 6 or hr == 12 or hr == 18:

            CHUNKS = {'time': 48, 'projection_x_coordinate': 150, 'projection_y_coordinate': 150, 
                      'time_2': 45, 'time_3': diffVar}
        else:
            CHUNKS = {'time': 18, 'projection_x_coordinate': 150, 'projection_y_coordinate': 150, 'time_1': 17}

        compressor = Blosc(cname = 'lz4', clevel = 9, shuffle = 1)

        del cube_list, cubes, new_cube

        dalist = zutils.cubelist_to_dalist(cubelist)
        ds0 = xr.merge(dalist,compat='override')
        ds = ds0.chunk(CHUNKS)

        print('dataset ready')

        for i in ds.data_vars:

            try:
                ds1 = ds[i].to_dataset()
                if i == 'surface/PRES':
                    encoding = {v: {'compressor': compressor, 'dtype': '<f4', '_FillValue': '-9999'} for v in ds1.data_vars}
                else:
                    encoding = {v: {'compressor': compressor, 'dtype': '<f2', '_FillValue': '-9999'} for v in ds1.data_vars}

                ds1.to_zarr(store = storeFcst, group=i, mode = 'w',encoding = encoding, consolidated = True)
            except:
                pass

        del ds, ds0

        t2 = time.time()
        print(t2-t1)

20210314
model run 0


ValueError: some chunks keys are not dimensions on this object: {'time_3'}

In [5]:
dataLoc = '/uufs/chpc.utah.edu/common/home/horel-group4/tmccorkle/hrrr_conus/'
dataStore = '/uufs/chpc.utah.edu/common/home/horel-group4/tmccorkle/hrrr_conus/'

month = '2021'
day = ['0314']
        
nDay = len(day)

idx = [0,48,54,72,90,108,126,162,180,198,216,234,252,288,306,324,342,
           360,378,414,432,450,468,486,504]

for d in range(0,nDay):
    dayList = month+day[d]
    print(dayList)
    forecastDir = dataLoc+dayList+'/forecasts/'
    writeTo = dataStore+dayList+'/'

    filesFcst = sorted(os.listdir(forecastDir))

    filepaths = [os.path.join(forecastDir, fileName) for fileName in filesFcst]

                                            
    for hr in range(0,1):
        t1 = time.time()
        print('model run',hr)
        hour = '{:02d}'.format(hr)

        storeFcst = zarr.DirectoryStore(writeTo+dayList+'_'+hour+'z_fcst.zarr')
        root_fcst = zarr.group(store=storeFcst)#,overwrite=True)
        
        
        a = idx[hr]
        z = idx[hr+1]

        hrs = (z-a)

        cubes = iris_grib.load_cubes(filepaths[a:z])
        cube_list = list(cubes)

        zarr_group = zutils.get_idx_forecast(dayList,hrs)
        cubelists = zutils.rename_cubes(cube_list, zarr_group)


        cubelists = iris.cube.CubeList(cubelists)
        new_cube = cubelists.merge(unique = False)

20210314
model run 0
